In [2]:
pip install cirq

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.25.3 which is incompatible.
tensorboard 2.11.0 requires protobuf<4,>=3.9.2, but you have protobuf 4.25.3 which is incompatible.
google-cloud-storage 1.31.0 requires google-auth<2.0dev,>=1.11.0, but you have google-auth 2.29.0 which is incompatible.
google-cloud-core 1.7.1 requires google-api-core<2.0.0dev,>=1.21.0, but you have google-api-core 2.18.0 which is incompatible.
google-cloud-core 1.7.1 requires google-auth<2.0dev,>=1.24.0, but you have google-auth 2.29.0 which is incompatible.


INFO: pip is looking at multiple versions of google-api-core[grpc] to determine which version is compatible with other requirements. This could take a while.
  Created wheel for lark: filename=lark-0.11.3-py2.py3-none-any.whl size=99646 sha256=05f5435ff94b900d2c4070710f99dd2b8b7045c3e732070b2519971b0f8437e5
  Stored in directory: c:\users\lenovo\appdata\local\pip\cache\wheels\ec\6a\24\f8eeaf52fee56bfe54309621b59c41bb7f1df56f4bfbcdb0ce


  Created wheel for rpcq: filename=rpcq-3.11.0-py3-none-any.whl size=45989 sha256=49d660bd00b744a0727fdce746d2d81ab334ac72571d45393d6a5c3390962e63
  Stored in directory: c:\users\lenovo\appdata\local\pip\cache\wheels\a6\c4\42\34581dfe489802146924ad802b13aa7fe3820f9e8c15f67afc
Successfully built lark rpcq
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.1
    Uninstalling protobuf-3.19.1:
      Successfully uninstalled protobuf-3.19.1
  Attempting uninstall: cachetools
    Found existing installation: cachetools 4.2.2
    Uninstalling cachetools-4.2.2:
      Successfully uninstalled cachetools-4.2.2
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.7.1
    Uninstalling pydantic-2.7.1:
      Successfully uninstalled pydantic-2.7.1
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.42.0
    Uninstalling grpcio-1.42.0:
      Successfully uninstalled grpcio-1.42.0
  Attempting uninstall: googleapis-common-protos

In [10]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    import cirq

    print("installed cirq.")


import cirq
import numpy as np
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

def alice_circuit(qubits, basis):
    """Alice generates a random bit string and encodes it onto qubits."""
    random_bits = np.random.randint(2, size=len(qubits))
    for i, bit in enumerate(random_bits):
        if bit == 1:
            if basis[i] == 0:  # Prepare qubit in Z basis
                yield cirq.X(qubits[i])
            else:  # Prepare qubit in X basis
                yield cirq.H(qubits[i])

def bob_measurement(qubits, basis):
    """Bob measures the qubits in the specified basis."""
    measurements = []
    for i, qubit in enumerate(qubits):
        if basis[i] == 0:  # Measure in Z basis
            yield cirq.measure(qubit, key=f'B{i}')
        else:  # Measure in X basis
            yield cirq.H(qubit)
            yield cirq.measure(qubit, key=f'B{i}')
            yield cirq.H(qubit)

def aes_encrypt(key, plaintext):
    """Encrypts plaintext using AES encryption with padding."""
    backend = default_backend()
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=backend)
    encryptor = cipher.encryptor()
    
    # Add padding to the plaintext to ensure its length is a multiple of the block size
    padding_length = 16 - (len(plaintext) % 16)
    padded_plaintext = plaintext + bytes([padding_length]) * padding_length
    
    ciphertext = encryptor.update(padded_plaintext) + encryptor.finalize()
    return ciphertext

def aes_decrypt(key, ciphertext):
    """Decrypts ciphertext using AES decryption and removes padding."""
    backend = default_backend()
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=backend)
    decryptor = cipher.decryptor()
    decrypted_data = decryptor.update(ciphertext) + decryptor.finalize()
    
    # Remove padding from the decrypted data
    padding_length = decrypted_data[-1]
    decrypted_data = decrypted_data[:-padding_length]
    
    return decrypted_data

def main():
    # Number of qubits to use for the key
    num_qubits = 10
    
    # Alice prepares qubits and sends them to Bob
    alice_qubits = cirq.LineQubit.range(num_qubits)
    alice_basis = np.random.randint(2, size=num_qubits)  # Randomly choose X or Z basis
    alice_circuit_gen = alice_circuit(alice_qubits, alice_basis)
    
    # Bob measures the qubits in random bases
    bob_qubits = cirq.LineQubit.range(num_qubits)
    bob_basis = np.random.randint(2, size=num_qubits)  # Randomly choose X or Z basis for measurement
    bob_measurement_gen = bob_measurement(bob_qubits, bob_basis)
    
    # Simulate the quantum circuit
    circuit = cirq.Circuit()
    circuit.append(alice_circuit_gen)
    circuit.append(bob_measurement_gen)
    
    # Simulate the circuit
    sim = cirq.Simulator()
    results = sim.run(circuit)
    
    # Alice and Bob compare basis
    shared_key = ''
    for i in range(num_qubits):
        if alice_basis[i] == bob_basis[i]:
            shared_key += str(results.measurements[f'B{i}'][0][0])
    
    print("Shared Key:", shared_key)
    
    # Use the shared key for AES encryption
    key = bytes.fromhex(shared_key.zfill(32))[:16]  # Use first 128 bits of the shared key
    plaintext = b"Hello, Bob! This is a secret message. i will kill you"
    ciphertext = aes_encrypt(key, plaintext)
    
    print("Encrypted Message:", ciphertext.hex())
    
    # Decrypt the ciphertext using AES decryption
    decrypted_message = aes_decrypt(key, ciphertext)
    
    print("Decrypted Message:", decrypted_message.decode('utf-8'))

if __name__ == "__main__":
    main()

Shared Key: 10010
Encrypted Message: 5d7924babf33423e5a73a995eb53679bd74a76482b54799fe6ffda5d5b38168745d4d5c4d8f16d624d9554b53bd2885dc4f1a9cec72378bdff6c131209bf530b
Decrypted Message: Hello, Bob! This is a secret message. i will kill you


In [8]:
def write_binary_file(filename, data):
    """Writes binary data to a file."""
    with open(filename, 'wb') as file:
        file.write(data)


In [9]:
def main():
    # Number of qubits to use for the key
    num_qubits = 10
    
    # Alice prepares qubits and sends them to Bob
    alice_qubits = cirq.LineQubit.range(num_qubits)
    alice_basis = np.random.randint(2, size=num_qubits)  # Randomly choose X or Z basis
    alice_circuit_gen = alice_circuit(alice_qubits, alice_basis)
    
    # Bob measures the qubits in random bases
    bob_qubits = cirq.LineQubit.range(num_qubits)
    bob_basis = np.random.randint(2, size=num_qubits)  # Randomly choose X or Z basis for measurement
    bob_measurement_gen = bob_measurement(bob_qubits, bob_basis)
    
    # Simulate the quantum circuit
    circuit = cirq.Circuit()
    circuit.append(alice_circuit_gen)
    circuit.append(bob_measurement_gen)
    
    # Simulate the circuit
    sim = cirq.Simulator()
    results = sim.run(circuit)
    
    # Alice and Bob compare basis
    shared_key = ''
    for i in range(num_qubits):
        if alice_basis[i] == bob_basis[i]:
            shared_key += str(results.measurements[f'B{i}'][0][0])
    
    print("Shared Key:", shared_key)
    
    # Write shared key to file
    write_file("shared_key.txt", shared_key.encode('utf-8'))
    
    # Use the shared key for AES encryption
    key = bytes.fromhex(shared_key.zfill(32))[:16]  # Use first 128 bits of the shared key
    
    # Read plaintext message from file
    plaintext = read_file("plaintext_message.txt")
    
    # Encrypt the plaintext message
    ciphertext = aes_encrypt(key, plaintext)
    
    # Write encrypted message to file
    write_binary_file("encrypted_message.txt", ciphertext)  # Modified function for writing binary file
    
    print("Encrypted Message:", ciphertext.hex())

if __name__ == "__main__":
    main()


Shared Key: 0000
Encrypted Message: a891af081ea8fd3e810ce4561b67bec8086d8eec7ec41bd7bc8e32be2d25de64


In [11]:
def read_binary_file(filename):
    """Reads binary data from a file."""
    with open(filename, 'rb') as file:
        data = file.read()
    return data


In [12]:
def main():
    # Number of qubits to use for the key
    num_qubits = 10
    
    # Bob measures the qubits in random bases
    bob_qubits = cirq.LineQubit.range(num_qubits)
    bob_basis = np.random.randint(2, size=num_qubits)  # Randomly choose X or Z basis for measurement
    bob_measurement_gen = bob_measurement(bob_qubits, bob_basis)
    
    # Simulate the quantum circuit
    circuit = cirq.Circuit()
    circuit.append(bob_measurement_gen)
    
    # Simulate the circuit
    sim = cirq.Simulator()
    results = sim.run(circuit)
    
    # Read shared key from file
    shared_key = read_file("shared_key.txt").decode('utf-8')
    
    # Use the shared key for AES decryption
    key = bytes.fromhex(shared_key.zfill(32))[:16]  # Use first 128 bits of the shared key
    
    # Read encrypted message from file
    ciphertext = read_binary_file("encrypted_message.txt")  # Modified function for reading binary file
    
    # Decrypt the ciphertext message
    decrypted_message = aes_decrypt(key, ciphertext)
    
    # Write decrypted message to file
    write_file("decrypted_message.txt", decrypted_message)
    
    print("Decrypted Message:", decrypted_message.decode('utf-8'))

if __name__ == "__main__":
    main()


Decrypted Message: this is a secret message
